In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
CSV_PATH = "socal2.csv"
IMAGE_DIR = "socal_pics"
BATCH_SIZE = 16
EPOCHS = 5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
df = pd.read_csv(CSV_PATH)

In [6]:
print(df.columns)
target = "price"
image_id_col = "image_id"

# Target
y = df[target].values

# Drop non-numeric + target
X_tab = df.drop(columns=[target, image_id_col, "street", "citi"])

# Ensure numeric only
X_tab = X_tab.select_dtypes(include=["int64", "float64"])

scaler = StandardScaler()
X_tab_scaled = scaler.fit_transform(X_tab)

# Image IDs (used later in Dataset)
image_ids = df[image_id_col].values


Index(['image_id', 'street', 'citi', 'n_citi', 'bed', 'bath', 'sqft', 'price'], dtype='object')


In [7]:
X_train_tab, X_test_tab, y_train, y_test, train_ids, test_ids = train_test_split(
    X_tab_scaled, y, df[image_id_col].values, test_size=0.2, random_state=42
)

In [8]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [9]:
class HouseDataset(Dataset):
    def __init__(self, tabular, prices, ids, img_dir):
        self.tabular = torch.tensor(tabular, dtype=torch.float32)
        self.prices = torch.tensor(prices, dtype=torch.float32)
        self.ids = ids
        self.img_dir = img_dir

    def __len__(self):
        return len(self.prices)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, str(self.ids[idx]).strip() + ".jpg")
        if not os.path.exists(img_path):
            image = torch.zeros(3, 224, 224)  # black dummy image
        else:
            image = Image.open(img_path).convert("RGB")
            image = transform(image)
        return image, self.tabular[idx], self.prices[idx]


In [10]:
train_ds = HouseDataset(X_train_tab, y_train, train_ids, IMAGE_DIR)
test_ds = HouseDataset(X_test_tab, y_test, test_ids, IMAGE_DIR)

In [11]:
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

In [12]:
class MultiModalModel(nn.Module):
    def __init__(self, tabular_dim):
        super().__init__()

        self.cnn = models.resnet18(weights=None)
        self.cnn.fc = nn.Identity()

        self.tabular_net = nn.Sequential(
            nn.Linear(tabular_dim, 64),
            nn.ReLU()
        )

        self.fc = nn.Sequential(
            nn.Linear(512 + 64, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, image, tabular):
        img_feat = self.cnn(image)
        tab_feat = self.tabular_net(tabular)
        x = torch.cat([img_feat, tab_feat], dim=1)
        return self.fc(x).squeeze()


In [13]:
model = MultiModalModel(X_train_tab.shape[1]).to(DEVICE)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [14]:
for epoch in range(EPOCHS):
    model.train()
    for imgs, tabs, prices in train_loader:
        imgs, tabs, prices = imgs.to(DEVICE), tabs.to(DEVICE), prices.to(DEVICE)
        optimizer.zero_grad()
        preds = model(imgs, tabs)
        loss = criterion(preds, prices)
        loss.backward()
        optimizer.step()

In [16]:
model.eval()
preds_list = []
true_list = []

with torch.no_grad():
    for imgs, tabs, prices in test_loader:
        imgs = imgs.to(DEVICE)
        tabs = tabs.to(DEVICE)

        preds = model(imgs, tabs)

        preds_list.extend(preds.cpu().numpy())
        true_list.extend(prices.numpy())

# Metrics (sklearn old version compatible)
mae = mean_absolute_error(true_list, preds_list)

mse = mean_squared_error(true_list, preds_list)
rmse = np.sqrt(mse)

print("MAE:", mae)
print("RMSE:", rmse)

MAE: 705352.2349019336
RMSE: 802802.165460924


In [ ]:
# pip install torchvision